# Set up Feature Monitoring for a Feature Group

In [21]:
# Establish connection with the feature store

import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/15484
Connected. Call `.close()` to terminate connection gracefully.


In [22]:
# Retrieve an the "price" feature group
trans_fg = fs.get_feature_group("price", version=1)

In [23]:
# compute statistics for all the features in the feature group
fg_full_monitoring = trans_fg.create_statistics_monitoring(
    name="fg_full_monitoring",
    description="Compute statistics on all data of all features of the Feature Group on a daily basis",
).save()

In [24]:
# Compute statistics on a reduced share of the data
fg_full_monitoring.detection_window_config.row_percentage = 0.1

In [25]:
fg_full_monitoring_sliding = trans_fg.create_statistics_monitoring(
    name="fg_full_monitoring_sliding",
    description="Compute statistics on 20% of the data inserted in the last week of all features of the Feature Group on a weekly basis",
    cron_expression="0 0 12 ? * MON *",
).with_detection_window(
    time_offset="1w", # fetch data from the last week
    row_percentage=0.2,
).save()

In [26]:
fg_price_monitoring_reference_value = trans_fg.create_feature_monitoring(
    name="fg_price_monitoring_reference_value",
    feature_name="price",
    cron_expression="0 0 12 ? * MON *",
    description="Compute descriptive statistics on the amount Feature in the last week of data inserted in the Feature Group",
).with_detection_window(
    time_offset="1w", # fetch data from the last week
    row_percentage=0.2,
)

In [27]:
fg_price_monitoring_reference_value.with_reference_value(
    value=100,
).compare_on(
    metric="mean",
    threshold=50,
).save()

FeatureMonitoringConfig('fg_price_monitoring_reference_value', STATISTICS_COMPARISON)

In [30]:
price_fv = fs.get_feature_view("price_fv", version=1)

fv_price_monitoring_reference_td = price_fv.create_feature_monitoring(
    name="fv_price_monitoring_reference_td",
    feature_name="price",
    cron_expression="0 0 12 ? * * *",
    description="Compute and compare descriptive statistics on the amount Feature in the last hour of data inserted in the Feature View",
).with_detection_window(
    time_offset="1h", # fetch data from the last hour
    row_percentage=0.2,
).with_reference_training_dataset(
    training_dataset_version=1, # use the training dataset used to train your production model
).compare_on(
    metric="mean",
    threshold=50,
).save()